## Classical SDP relaxation of OPF
As a first step, we present a simple implementation of the classical SDP formulation of the Optimnal Power Flow problem using CVXPY.

### Setup
We start by importing pandapower for the test cases;
CVXPY, an open source Python-embedded modeling language for convex optimization problems;
MOSEK, a large scale optimization software. 

In [1]:
import pandapower.networks as nw
import pandapower as pp
import numpy as np
import cvxpy as cp
from scipy.sparse import csr_matrix
import mosek
import plotly.express as px

### Load Network Data
Now,let's make a function that loads the IEEE test cases from Pandapower.

In [33]:
# Select IEEE test case
def load_network(case):
    if case == 14:
        return nw.case14()
    elif case == 30:
        return nw.case30()
    elif case == 57:
        return nw.case57()
    elif case == 118:
        return nw.case118()
    elif case == 300:
        return nw.case300()
    elif case == 'GB':
        return nw.GBnetwork()
    else:
        raise ValueError("Unsupported test case")

In [34]:
# Load the chosen IEEE test case
case_number = 'GB' # Change to 30, 57, or 118 for larger cases
net = load_network(case_number)

pp.runpp(net)  # Run power flow to initialize values

When a power flow is carried out, the element based grid model is translated into a bus-branch model. That bus-branch model is stored in a data structure that is based on the PYPOWER/MATPOWER casefile (with some extensions). This ppc can be accessed after power flow using net._ppc
We will get the bus admittance matrix using exactly that and extract all the parameters we need.

In [35]:

#make sure all units are consistent (convert everything to per unit)
Sbase = net.sn_mva # unit value for power MVA
Vbase = 135 # unit value for voltage kV
Zbase = Vbase**2 / Sbase # base impedance in Ohms
Ybase = 1 / Zbase # base admittance in S

#Now, we extract network parameters
Ybus = net._ppc["internal"]["Ybus"].todense() # Bus admittance matrix (Already in per unit. We need todense since the matrices are stored as sparcse matrices, but dense are easier to work with)
n = len(net.bus)  # Number of buses
# slack_bus = net.gen.bus.values[0]  # Index of the slack bus
branch = net.line  # Get each transmission line's specification for line constraints
#we need to add line flow constraints

# Extract generator information for generation constraints
gen_buses = net.gen["bus"].values #index of the generators.
min_p_mw = net.gen["min_p_mw"].values #minimal real power
max_p_mw = net.gen["max_p_mw"].values
min_q_mw = net.gen["min_q_mvar"].values #minimal reactive power
max_q_mw = net.gen["max_q_mvar"].values

# Extract load buses for generation constraints
load_buses = net.load["bus"].values #index of the load buses.
#load active and reactive power for load
p_load = np.zeros(n)
q_load = np.zeros(n)
for i,row in net.load.iterrows():
    p_load[row['bus']] = -row['p_mw'] #negative sign because pandapower defines load as positive, but we need it as negative for consumption
    q_load[row['bus']] = -row['q_mvar']

# Voltage limits
min_v_pu = net.bus["min_vm_pu"].values ** 2  # Squared for SDP
max_v_pu = net.bus["max_vm_pu"].values ** 2

# Slack bus voltage
vm_slack = net.ext_grid['vm_pu'][0] #initial voltage of the slack

# Extract generator cost coefficients from net.poly_cost
cost = net.poly_cost.drop([0])
c2 = cost["cp2_eur_per_mw2"].values   # Quadratic cost coefficients
c1 = cost["cp1_eur_per_mw"].values   # Linear cost coefficients
c0 = cost["cp0_eur"].values         # Constant cost coefficients

# Ensure the cost coefficients have the correct size
if len(c2) != len(gen_buses):
    print("The number of generators does not match the number of cost coefficients")
elif len(c1) != len(gen_buses):
    print("The number of generators does not match the number of cost coefficients")
elif len(c0) != len(gen_buses):
    print("The number of generators does not match the number of cost coefficients")
else:
    print("The number of generators matches the number of cost coefficients")


# # Make the cost matrix
# C = np.zeros((n, n))
# for i, gen_bus in enumerate(gen_buses):
#     C[gen_bus, gen_bus] = c2[i]

# # Print the cost matrix
# np.set_printoptions(precision=10, suppress=True)
# print(C)

The number of generators matches the number of cost coefficients


### Define Variables and the objective function

The objective function of standard Optimal Power Flow (OPF) problem is:
$$\min \space \sum_{j \in G} f_j(P_{j})$$
where $f_j(P_{j})$ is the cost function of choice. Traditionally, the following cost function has been used:
$$\min \space \sum_{j \in G}( c_{j2}P_j^2 + c_{j1}P_j + c_{jo})$$
For each bus $j$ in the grid $G$:
$P_j = power \space generation, \space c_{j2} = quadratic \space cost \space  coefficient, \space c_{j1} = linear \space cost \space  coefficient, \space c_{j0} = constant \space cost.$

The objective function is then subjected to a set of constraints.

Semidefinite Programming (SDP) is one of the conic form problems, which all have a linear objective function and convex inequality/equality constraints (that could be quadratic). SDP in particular tackles the cone of positive semidefinite n x n matrices, W. The SDP Standard Form objective function is as follows:

$$min \space \space tr(CW)$$
subjected to $W \succcurlyeq 0$ and other linear constraints, which are

**Semidefinite positivity**
$$W\succcurlyeq 0$$

**Active Power Flow**
$$p_{min, j} \le tr (\Phi_jW_B) \le p_{max, j}$$
where $\Phi_j = \frac{1}{2}(Y^H_j + Y_j)$

**Reactive Power Flow**
$$q_{min, j} \le tr (\Psi_jW_B) \le q_{max, j}$$
where $\Psi_j = \frac{1}{2i}(Y^H_j + Y_j)$

**Voltage Magnitude Constraints**
$$v_{min, j} \le tr (J_jW_B) \le v_{max, j}$$
where J is the Hermitian matrix with a single 1 in the (j,j)th entry and zero everywhere else.

To transform the quadratic objective function into SDP, we can do the following:

#### Attempt 1

Let $$C_j := \begin{bmatrix}c_{j2} & c_{j1}/2 \\c_{j1}/2 & c_{j0}\end{bmatrix},\space X_j := \begin{bmatrix}a_j & P_j \\P_j & 1\end{bmatrix}$$
where $a_j$ is a new variable meant to represent $P_j^2$. Instead of enforcing an equality, we will relax it into a matrix inequality. We add the following constraint:
$$\begin{bmatrix}a_j & P_j \\P_j & 1\end{bmatrix} \succcurlyeq 0 $$
By Schur’s complement condition, the constraint above is equivalent to
$$P^2_j \le a_j$$
As we optimize, the solver will push the above inequality toward equality. Now, we can formulate the objective function as:
$$min \space \sum_{j \in G} tr(C_jX_j) $$
which is equivalent to $$\min \space \sum_{j \in G}( c_{j2}P_j^2 + c_{j1}P_j + c_{jo})$$

In [ ]:
# Define an arbitrary n x n Hermitian W (Directly enforcing W= V V^+ will require X to be rank-1, which is too strict. We use the power balance, PSD and voltage constraints to guide this arbitrary X to X = VV^+)
W = cp.Variable((n, n), hermitian=True)

# Define phi and psi matrices for each bus as the constraints
Phi = [] # Hermitian part of Y_j
Psi = [] # Skewed Hermitian part of Y_j
J = [] # Hermitian matrix with a single 1 in the (j,j)th entry and zero everywhere else.
for j in range(n):
    Y_j = np.zeros_like(Ybus,dtype=complex) #Y_j is the admittance matrix with only the j-th row, everything else is zero
    Y_j[j,:] = Ybus[j,:]
    Y_j_H = np.conj(Y_j).T #Hermitian of Y_j
    
    J_j= np.zeros_like(Ybus,dtype=complex)
    J_j[j,j] = 1

    Phi_j = 1/2 * (Y_j_H + Y_j) 
    Psi_j = 1/(2j) * (Y_j_H - Y_j)

    Phi.append(Phi_j) 
    Psi.append(Psi_j)
    J.append(J_j)

# Objective Function Formulation in Standard SDP Form
################################################################################
# # Attempt 0
# # Initialize 
# C = []
# X = []

# for i in range(len(gen_buses)):
#     # Real power generation at generator i
#     a = cp.real(cp.trace(Phi[gen_buses[i]] @ W)) * Sbase

#     # Define the cost matrix for generator i
#     C_i = cp.Constant(np.array([[c2[i], 0], [0, c1[i]]]))
#     C.append(C_i)

#     # Define the variable matrix for generator i
#     X_i = cp.vstack([
#         cp.hstack([a**2, 0]),
#         cp.hstack([0, a])
#     ])
#     X.append(X_i)

# objective_terms = cp.sum([
#     cp.trace(C[i] @ X[i]) for i in range(len(gen_buses))
# ])

# objective = cp.Minimize(objective_terms)

# ############################################################################

# Attempt 1
# Initialize 
C = []
X = []

for i in range(len(gen_buses)):
    # Real power generation at generator i
    P_j = cp.real(cp.trace(Phi[gen_buses[i]] @ W)) * Sbase
    # We want a = P^2, and we will add a psd constraint later to make it happen
    a = cp.Variable()

    # Define the cost matrix for generator i
    C_i = cp.Constant(np.array([[c2[i], c1[i]/2], [c1[i]/2, c0[i]]]))
    C.append(C_i)

    # Define the variable matrix for generator i
    X_i = cp.vstack([
        cp.hstack([a, P_j]),
        cp.hstack([P_j, 1])
    ])
    X.append(X_i)

objective_terms = cp.sum([
    cp.trace(C[i] @ X[i]) for i in range(len(gen_buses))
])

objective = cp.Minimize(objective_terms)



##############################################################################

# Constraints
constraints = []

for j in range(n):
    if j == 0:
        # For the slack bus, we fix the voltage magnitude
        constraints.append(cp.abs(cp.real(cp.trace(J[j] @ W))-(vm_slack)**2) <= 1e-5)
    else:

        # Voltage magnitude constraints
        constraints.append(min_v_pu[j]<=cp.real(cp.trace(J[j] @ W)))
        constraints.append(cp.real(cp.trace(J[j] @ W))<=max_v_pu[j])
    
    # #Change of variables
    # constraints.append(a[j] == cp.real(cp.trace(Phi[j] @ W)))*Sbase

    # Power balance equations (real and reactive)
    Pin_j = cp.trace(Phi[j]@W)*Sbase
    Qin_j = cp.trace(Psi[j]@W)*Sbase

    # If bus is a generator bus, enforce generation limits
    if j in gen_buses:
        idx = np.where(gen_buses == j)[0][0]  # Get generator index
        constraints.append(min_p_mw[idx] <= cp.real(Pin_j))
        constraints.append(cp.real(Pin_j) <= max_p_mw[idx])
        constraints.append(min_q_mw[idx] <= cp.real(Qin_j))
        constraints.append(cp.real(Qin_j) <= max_q_mw[idx])
        #add psd constraint to make a = P^2
        constraints.append(X[idx] >> 0)
    else:
        # For non-generator buses, enforce power balance constraints
        constraints.append(cp.real(Pin_j) == p_load[j])
        constraints.append(cp.real(Qin_j) == q_load[j])

# Positive semidefinite constraint
constraints.append(W >> 0)

### Solve using cvxpy's SDP solver

In [ ]:
# Solve the SDP
prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.MOSEK)

# Print constraints
print("p min (MW):", min_p_mw)
print("p max:(MW)", max_p_mw)
print("q min (MW):", min_q_mw)
print("q max (MW):", max_q_mw)

# Print results
print("Problem Status:", prob.status)
print("Optimal cost EUR:", prob.value)
P_g_sdp = [cp.real(cp.trace(Phi[j] @ W)).value * Sbase for j in gen_buses]
print("Real power generation in MW (SDP OPF):", P_g_sdp)
Q_g_sdp = [cp.imag(cp.trace(Psi[j] @ W)).value * Sbase for j in gen_buses]
print("Reactive power generation in MW (SDP OPF):", Q_g_sdp)
V_sdp = [np.sqrt(cp.real(W[j, j]).value) for j in range(n)]
print("Voltage magnitudes in pu(SDP OPF):", V_sdp)

/home/catz0449/VSDP/env/lib/python3.12/site-packages/mosek/__init__.py:18617: UserWarning:

Argument sub in putvarboundlist: Incorrect array format causing data to be copied

/home/catz0449/VSDP/env/lib/python3.12/site-packages/mosek/__init__.py:18925: UserWarning:

Argument subj in putclist: Incorrect array format causing data to be copied

/home/catz0449/VSDP/env/lib/python3.12/site-packages/mosek/__init__.py:18349: UserWarning:

Argument sub in putconboundlist: Incorrect array format causing data to be copied



p min (MW): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
p max:(MW) [100. 100. 100. 100. 550. 185. 100. 100. 100. 100. 320. 414. 100. 107.
 100. 100. 100. 100. 100. 119. 304. 148. 100. 100. 255. 260. 100. 491.
 492. 100. 100. 100. 100. 100. 100. 577. 100. 104. 707. 100. 100. 100.
 100. 352. 140. 100. 100. 100. 100. 136. 100. 100. 100.]
q min (MW): [   -5.  -300.   -13.  -300.  -147.   -35.   -10.   -16.    -8.  -300.
   -47. -1000.  -300.  -300.   -14.    -8.    -8.  -300.  -300.  -100.
   -85.  -300.    -8.    -8.   -60.  -100.   -20.   -67.   -67.   -10.
  -100.  -100.    -6.    -8.   -20.  -165.    -8.  -100.  -210.  -300.
  -100.    -3.  -100.   -50.   -15.    -8.    -8.  -200.    -8.  -100.
  -100.  -100. -1000.]
q max (MW): [  15.  300.   50.  300.  200.  120.   30.   50.   24.  300.  140. 1000.
  300.  300.   42.   24.   24.  300.  300.  100.  210.  300.   23.   

### Check whether the solution is physically feasible
The semidefinite relaxation makes OPF convex, but also opens the door to:

1. Inexact solutions (i.e., WW is not rank-1)

2. Unrealistically cheap solutions that cannot occur in real networks.

Let's check our solution.

In [28]:
#Check the rank of W
eigvals = np.linalg.eigvalsh(W.value)
rank_W = np.sum(eigvals > 1e-6)  # numerical rank threshold
print("Rank of W:", rank_W)

# Check if the solution is feasible
for i in range(len(P_g_sdp)):
    if P_g_sdp[i] < min_p_mw[i]:
        print(f"Generation at bus {i} is below the minimum limit.")
    elif i > max_p_mw[i]:
        print(f"Generation at bus {i} is above the maximum limit.")

# Check if the voltage magnitudes are within limits
for i in range(n):
    if V_sdp[i]**2 < min_v_pu[i]:
        print(f"Voltage at bus {i} is below the minimum limit.")
    elif V_sdp[i]**2 > max_v_pu[i]:
        print(f"Voltage at bus {i} is above the maximum limit.")

# Check if the power balance equations are satisfied
for j in range(n):
    Pin_j = cp.trace(Phi[j] @ W).value * Sbase
    Qin_j = cp.trace(Psi[j] @ W).value * Sbase
    if j in gen_buses:
        idx = np.where(gen_buses == j)[0][0]  # Get generator index
        if Pin_j < min_p_mw[idx]:
            print(f"Power balance at bus {j} is below the minimum limit.")
        elif Pin_j > max_p_mw[idx]:
            print(f"Power balance at bus {j} is above the maximum limit.")
    # If bus is a load bus, check if the power balance is approximately equal to the load
    elif abs(Pin_j - p_load[j]) > 1e-5:
            print(f"Power balance at bus {j} does not match load.")



Rank of W: 7


### Benchmark using Classical OPF

In [29]:
pp.runopp(net)

#Print results
opf_cost = net.res_cost
print("OPF Cost ($/hr):", opf_cost)
P_g_opf = net.res_gen["p_mw"].values
print("OPF Real power generation (OPF):", P_g_opf)
Q_g_opf = net.res_gen["q_mvar"].values
print("OPF Reactive power generation (OPF):", Q_g_opf)
V_opf = net.res_bus["vm_pu"].values
print("OPF Voltage magnitudes (OPF):", V_opf)

OPF Cost ($/hr): 129695.10150252415
OPF Real power generation (OPF): [2.64353041e+01 2.19743883e-05 4.49970318e-02 1.89600639e-05
 4.01839324e+02 8.57867413e+01 2.08390182e+01 1.31945571e+01
 2.15588758e+01 2.07195881e-05 1.93855161e+02 2.79800980e+02
 1.00275054e+01 7.24886947e+00 1.49659279e+01 4.72709100e+00
 1.05172891e+01 4.92773379e+01 4.10038604e+01 1.90408632e+01
 1.93268724e+02 4.95340169e+01 3.19518183e+01 3.23860854e+01
 1.49587532e+02 1.48281257e+02 1.13750085e-05 3.51883563e+02
 3.48622729e+02 5.83914033e-05 6.54044608e-05 1.01401090e-04
 1.82694998e+01 2.40492705e+01 1.37048240e-05 4.30696079e+02
 6.53422594e-06 3.62960551e+00 5.02344896e+02 8.51876943e-06
 7.76958763e-06 6.07311431e-06 1.07192905e-05 2.31308301e+02
 3.82505372e+01 2.47999255e-04 5.44353369e+00 2.92709132e+01
 7.16511856e+00 3.52388596e+01 3.66689560e+01 7.22985703e-05
 7.09311838e-06]
OPF Reactive power generation (OPF): [  14.99994863   71.22792003   33.25270819  -78.25405012 -100.03304894
   54.4327306

### Comparison

In [30]:
print("SDP OPF Optimal cost (EUR):", prob.value)
print("OPF Cost (EUR):", opf_cost*0.87)

SDP OPF Optimal cost (EUR): 34717.45018787363
OPF Cost (EUR): 112834.738307196


In [31]:
import plotly.graph_objects as go

# Generator power plot
gen_fig = go.Figure()
gen_fig.add_trace(go.Scatter(x=gen_buses, y=P_g_sdp, mode='markers+lines', name='SDP OPF Gen (MW)'))
gen_fig.add_trace(go.Scatter(x=gen_buses, y=P_g_opf, mode='markers+lines', name='OPF Gen (MW)'))
gen_fig.update_layout(title="Generator Real Power Comparison", xaxis_title="Generator Bus", yaxis_title="P (MW)")
gen_fig.show()

In [32]:
# Voltage Magnitude plot
gen_fig = go.Figure()
gen_fig.add_trace(go.Scatter(x=list(range(n)), y=V_sdp, mode='markers+lines', name='SDP OPF V (pu)'))
gen_fig.add_trace(go.Scatter(x=list(range(n)), y=V_opf, mode='markers+lines', name='OPF V (pu)'))
gen_fig.update_layout(title="Voltage Magnitude Comparison", xaxis_title="Bus", yaxis_title="V (pu)")
gen_fig.show()